In [22]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.utils.data
from PIL import Image
from torch.utils.data.dataset import Dataset
import numpy as np
import pandas as pd

from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import pickle

In [23]:
num_classes = 120
batch_size = 5
epochs = 15
sample_submission = pd.read_csv('./data/sample_submission.csv')

In [24]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
train_transform = transforms.Compose([transforms.RandomResizedCrop(224),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                normalize])
valid_transform = transforms.Compose([transforms.Resize(256),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                normalize])

trainset = torchvision.datasets.ImageFolder("./data/train/", train_transform);
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

validset = torchvision.datasets.ImageFolder("./data/valid/", valid_transform);
validloader = torch.utils.data.DataLoader(validset, batch_size=batch_size,
                                          shuffle=False, num_workers=2)

classes = trainloader.dataset.classes
pickle.dump(classes, open("dog_breeds_labels.pickle", "wb"), 2)

In [25]:
def train(model, train, valid, optimizer, criterion, epochs=1):
    for epoch in range(epochs):
        print('Epoch ', epoch + 1, '/', epochs)
        
        running_loss = 0.
        running_corrects = 0.
        running_batches = 0.
       
        model.train()
        for i, (input, target) in enumerate(train):
            input_var = torch.autograd.Variable(input)
            target_var = torch.autograd.Variable(target)

            optimizer.zero_grad()

            output = model(input_var)
            if type(output) == tuple:
                output, _ = output
            _, preds = torch.max(output.data, 1)
            loss = criterion(output, target_var)

            loss.backward()
            optimizer.step()
            
            running_loss += loss.data[0]
            running_corrects += torch.sum(preds == target)
            running_batches += 1.
            
            ''' for testing 
            if i > 10:
                break
            '''

            print('\r', 'Batch', i, 'Loss', loss.data[0], end='')
            
        train_loss = running_loss / running_batches
        train_acc = running_corrects / len(train.dataset)
        print('\r', "Train Loss", train_loss, "Train Accuracy", train_acc)
            
        running_loss = 0.
        running_corrects = 0.
        running_batches = 0.

        model.eval()
        for i, (input, target) in enumerate(valid):
            input_var = torch.autograd.Variable(input, volatile=True)
            target_var = torch.autograd.Variable(target, volatile=True)

            output = model(input_var)
            if type(output) == tuple:
                output, _ = output
            _, preds = torch.max(output.data, 1)
            loss = criterion(output, target_var)

            running_loss += loss.data[0]
            running_corrects += torch.sum(preds == target)
            running_batches += 1.
            
            ''' for testing 
            if i > 10:
                break
            '''

        valid_loss = running_loss / running_batches
        valid_acc = running_corrects / len(valid.dataset)
        print('\r', "Val Loss", valid_loss, "Val Accuracy", valid_acc)

In [26]:
model = torchvision.models.squeezenet1_1(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

model.classifier = nn.Sequential(
    nn.Dropout(p=0.5),
    nn.Conv2d(512, num_classes, kernel_size=1),
    nn.ReLU(inplace=True),
    nn.AvgPool2d(13)
)
model.forward = lambda x: model.classifier(model.features(x)).view(x.size(0), num_classes)
for param in model.classifier.parameters():
    param.requires_grad = True
    
criterion = nn.CrossEntropyLoss()
#Optimize only the classifier
optimizer = torch.optim.SGD(list(filter(lambda p: p.requires_grad, model.parameters())), lr=0.001, momentum=0.9)

train(model, trainloader, validloader, optimizer, criterion, epochs=epochs)

Epoch  1 / 15
 Train Loss 3.729998452177888 Train Accuracy 0.20447634107772777
 Val Loss 2.8896977499088488 Val Accuracy 0.3448275862068966
Epoch  2 / 15
 Train Loss 2.9237218398634788 Train Accuracy 0.33146819121761345
 Val Loss 2.541808501864317 Val Accuracy 0.4032983508245877
Epoch  3 / 15
 Train Loss 2.6295154970488865 Train Accuracy 0.37842111665247535
 Val Loss 2.586290212803894 Val Accuracy 0.4352823588205897
Epoch  4 / 15
 Train Loss 2.5055620380629158 Train Accuracy 0.4077362851234643
 Val Loss 2.2854568257825125 Val Accuracy 0.4547726136931534
Epoch  5 / 15
 Train Loss 2.410577801014877 Train Accuracy 0.42598224060333295
 Val Loss 2.1819819955735444 Val Accuracy 0.4817591204397801
Epoch  6 / 15
 Train Loss 2.3097222640838666 Train Accuracy 0.4428901593480112
 Val Loss 2.1896566720635855 Val Accuracy 0.48125937031484256
Epoch  7 / 15
 Train Loss 2.2193444372423694 Train Accuracy 0.46417710740785795
 Val Loss 2.291605162139032 Val Accuracy 0.4867566216891554
Epoch  8 / 15
 Trai

In [27]:
torch.save(model.state_dict(), 'dog-breed-ident-squeezenet.pt')

In [28]:
class DogsData(Dataset):
    def __init__(self, root_dir, labels, transform, output_class=True):
        self.labels = labels
        self.root_dir = root_dir
        self.transform = transform
        self.output_class = output_class
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        item = self.labels.iloc[idx]
        path = '{}/{}.jpg'.format(self.root_dir, item['id'])
        image = Image.open(path).convert('RGB')

        if self.transform:
            image = self.transform(image)
            
        if not self.output_class:
            return image
        
        return image, item['class']

test_loader = torch.utils.data.DataLoader(
    DogsData('./data/test/x', sample_submission, valid_transform, output_class=False),
    batch_size=batch_size
)

In [29]:
model.load_state_dict(torch.load('dog-breed-ident-squeezenet.pt'))
model.eval()
results = []

for i, input in enumerate(test_loader):
    input_var = torch.autograd.Variable(input, volatile=True)
    output = model(input_var)
    results.append(F.softmax(output, dim=1).cpu().data.numpy())
    print('\r', 'Batch', i, end='')
    
    ''' for testing 
    if i > 10:
        break
    '''
        
results = np.concatenate(results)

 Batch 2071

In [30]:
results.shape

(10357, 120)

In [31]:
ids = sample_submission['id'].values
sample_df = pd.DataFrame(ids, columns=['id'])
#sample_df = sample_df[:48] #for testing only
#sample_df.shape
for index, breed in enumerate(classes):
    sample_df[breed] = results[:,index]

In [32]:
sample_df.to_csv('pred_squeezenet.csv', index=None)